In [1]:
import numpy as np
from serpentGenerator.functions.pin import pin
from serpentGenerator.data.materialLibrary import MATLIB
from serpentGenerator.functions.material import material
from serpentGenerator.functions.sqLattice import sqLat
from serpentGenerator.functions.hexLattice import hexLat
from serpentGenerator.functions.pinStack import pinStack
from serpentGenerator.functions.surfs import surfs
from serpentGenerator.functions.surf import surf
from serpentGenerator.functions.cell import cell
from serpentGenerator.functions.cells import cells
from serpentGenerator.functions.lats import lats
from serpentGenerator.functions.mix import mix
from serpentGenerator.functions.mats import mats
from serpentGenerator.functions.pins import pins
from serpentGenerator.functions.core import core
from serpentGenerator.functions.housing import housing
from serpentGenerator.functions.universe import universe
from serpentGenerator.functions.branches import branches
from serpentGenerator.functions.hexpin import hexpin

# Materials

In [2]:
fuel1 = MATLIB['UO2'].duplicateMat("fuelFA1")
fuel1.set('fractions', np.array([-0.04319337211, -0.83830401789, -.11850261])) # 4.9 % wt enrichment
fuel1.set('dens', -10.5216)
fuel1.set('rgb', "238 59 49")

fuel2 = fuel1.duplicateMat("fuelFA2")
fuel2.set('rgb', "255 165 0")

fuel3 = fuel1.duplicateMat("fuelFA3")
fuel3.set('rgb', "255 255 0")

fuel4 = fuel1.duplicateMat("fuelBlanket")
fuel4.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
fuel4.set('rgb', "255 255 255")

mod = MATLIB['H2O'].duplicateMat("water")
mod.set('rgb', "198 226 255")

clad = MATLIB['Zr'].duplicateMat("clad")
clad.set('rgb', "183 183 183")

ref = MATLIB['Reflector'].duplicateMat("reflector") # reflector plenum
ref.set('rgb', "204 153 255")

ba = MATLIB['B'].duplicateMat("B4C") # 90% wt enriched b-10

MB = 10.8
MC = 12
MB4C = MB*4 + MC
rhoB4C = 2.52
AvroNorm = .6022
nb4c = rhoB4C*AvroNorm/MB4C
nc12 = nb4c
b10_enrich = .90
b11_enrich = 1 - b10_enrich
nb10 = nb4c*b10_enrich
nb11 = nb4c*b11_enrich

b4cdens = -2.52
ba.set('rgb', "255 255 0")
ba.set('nuclides', np.array([5010, 5011, 6012]))
ba.set('fractions', np.array([nb10, nb11, nc12]))
ba.set('dens', -1*rhoB4C)
ba.set('isBurn', True)

# matList = mats()
# matList.addMats([fuel1, fuel2, fuel3, fuel4,  mod, clad, ref, ba])

# print(matList.toString())

# Axial Layers

1. Pin pitch (cm) 1.2623
2. UO2 pellet radius (cm) 0.4096
3. Zircaloy clad inner radius (cm) 0.4187
4. Zircaloy clad outer radius (cm) 0.4760
5. Guide thimble and instrumentation tube
6. Zircaloy inner radius (cm) 0.5531
7. Zircaloy outer radius (cm) 0.6133

In [3]:
ro_fuel = .4096
ro_clad = .4760
ri_gt = .5531
ro_gt = .6133

In [4]:
F1 = pin("F1", 3)
F1.set('materials', [fuel1, clad, mod])
F1.set('radii', [ro_fuel, ro_clad])

F2 = F1.duplicate("F2")
F2.set('materials', [fuel2, clad, mod])

F3 = F1.duplicate("F3")
F3.set('materials', [fuel3, clad, mod])

F4 = F1.duplicate("F4")
F4.set('materials', [fuel4, clad, mod])


G1 = pin("G1", 3)
G1.set('materials', [mod, clad, mod])
G1.set('radii', [ri_gt, ro_gt])

W1 = pin("W1", 3)
W1.set('materials', [mod])

RT = pin("RT", 1) #top reflector
RT.set('materials', [ref])

RB =  RT.duplicate("RB")

pinList1 = pins()
pinList1.addPins([F1, F2, F3, F4, W1, RT, RB, G1])


# Explicit BigT GuideThimble Cells

In [5]:
riClad = .5531
roClad = .6133

tClad = roClad - riClad
span1 = 70

gap1 = (360-span1*4)/4
theto1 = (90 - span1)/2
thetf1 = 360 - theto1

span2 = 55

gap2 = (360-span2*4)/4
theto2 = (90 - span2)/2
thetf2 = 360 - theto2

span3 = 60

gap3 = (360-span3*4)/4
theto3 = (90 - span3)/2
thetf3 = 360 - theto3

t1 = .090/10
t2 = .089/10
t3 = .019/10

riPad1 = riClad+ (tClad - t1)/2 
roPad1 = riPad1 + t1

riPad2 = riClad+ (tClad - t2)/2 
roPad2 = riPad2 + t2

riPad3 = riClad+ (tClad - t3)/2 
roPad3 = riPad3 + t3

vol1 = 4*(span1/360)*np.pi*(roPad1*roPad1 - riPad1*riPad1)*200
# print(vol1)

vol2 = 4*(span2/360)*np.pi*(roPad2*roPad2 - riPad2*riPad2)*200
# print(vol2)

vol3 = 4*(span3/360)*np.pi*(roPad3*roPad3 - riPad3*riPad3)*200
# print(vol3)

totvol = vol1*9*24 + vol2*12*24 + vol3*16*24
# print(totvol)

r1 = [riClad, riPad1, roPad1, roClad]
r2 = [riClad, riPad2, roPad2, roClad]
r3 = [riClad, riPad3, roPad3, roClad]

crs1 = [None]*len(r1)
baps1 = [None]*len(r1)
bapscc1 = [None]*len(r1)
clps1 = [None]*len(r1)
clpscc1 = [None]*len(r1)

crs2= [None]*len(r2)
baps2= [None]*len(r2)
bapscc2 = [None]*len(r2)
clps2= [None]*len(r2)
clpscc2= [None]*len(r2)

crs3= [None]*len(r3)
baps3= [None]*len(r3)
bapscc3 = [None]*len(r3)
clps3= [None]*len(r3)
clpscc3= [None]*len(r3)

alpha11= theto1
alpha12 = alpha11 + span1
alpha13 = thetf1
alpha14 = (alpha13 + gap1) % 360
                    
alpha21 = theto2
alpha22 = alpha21 + span2
alpha23 = thetf2
alpha24 = (alpha23 + gap2) % 360

alpha31 = theto3
alpha32 = alpha31 + span3
alpha33 = thetf3
alpha34 = (alpha33 + gap3) % 360

for i in range(0, len(r1)):
    crs1[i] = surf("crt1"+str(i+1), "cyl", np.array([0.0, 0.0, r1[i]]))
    baps1[i] = surf("bat1"+str(i+1), "pad", np.array([0.0, 0.0, riPad1, roPad1, alpha11, alpha12]))
    clps1[i] = surf("clt1"+str(i+1), "pad", np.array([0.0, 0.0, riPad1, roPad1, alpha13, alpha14]))
    bapscc1[i] = cell("bapadt1"+str(i+1), ba)
    bapscc1[i].setSurfs([baps1[i]], [1])
    clpscc1[i] = cell("clpadt1"+str(i+1), clad)
    clpscc1[i].setSurfs([clps1[i]], [1])
    alpha11 = alpha12 + gap1
    alpha12 = alpha11 + span1
    alpha13 = alpha14 + span1
    alpha14 = alpha13 + gap1
    
for i in range(0, len(r2)):
    crs2[i] = surf("crt2"+str(i+1), "cyl", np.array([0.0, 0.0, r2[i]]))
    baps2[i] = surf("bat2"+str(i+1), "pad", np.array([0.0, 0.0, riPad2, roPad2, alpha21, alpha22]))
    clps2[i] = surf("clt2"+str(i+1), "pad", np.array([0.0, 0.0, riPad2, roPad2, alpha23, alpha24]))
    bapscc2[i] = cell("bapadt2"+str(i+1), ba)
    bapscc2[i].setSurfs([baps2[i]],[1])
    clpscc2[i] = cell("clpadt2"+str(i+1), clad)
    clpscc2[i].setSurfs([clps2[i]],[1])
    alpha21 = alpha22 + gap2
    alpha22 = alpha21 + span2
    alpha23 = alpha24 + span2
    alpha24 = alpha23 + gap2
    
for i in range(0, len(r3)):
    crs3[i] = surf("crt3"+str(i+1), "cyl", np.array([0.0, 0.0, r3[i]]))
    baps3[i] = surf("bat3"+str(i+1), "pad", np.array([0.0, 0.0, riPad3, roPad3, alpha31, alpha32]))
    clps3[i] = surf("clt3"+str(i+1), "pad", np.array([0.0, 0.0, riPad3, roPad3, alpha33, alpha34]))
    bapscc3[i] = cell("bapadt3"+str(i+1), ba)
    bapscc3[i].setSurfs([baps3[i]],[1])
    clpscc3[i] = cell("clpadt3"+str(i+1), clad)
    clpscc3[i].setSurfs([clps3[i]],[1])
    alpha31 = alpha32 + gap3
    alpha32 = alpha31 + span3
    alpha33 = alpha34 + span3
    alpha34 = alpha33 + gap3
    
cc11 = cell("cc11", mod)
cc11.setSurfs([crs1[0]],[1])
cc12 = cell("cc12", clad)
cc12.setSurfs([crs1[0], crs1[1]],[0, 1])
cc13 = cell("cc13", clad)
cc13.setSurfs([crs1[2], crs1[3]],[0, 1])
cc14 = cell("cc14", mod)
cc14.setSurfs([crs1[3]], [0])


cc21 = cell("cc21", mod)
cc21.setSurfs([crs2[0]],[1])
cc22 = cell("cc22", clad)
cc22.setSurfs([crs2[0], crs2[1]],[0, 1])
cc23 = cell("cc23", clad)
cc23.setSurfs([crs2[2], crs2[3]],[0, 1])
cc24 = cell("cc24", mod)
cc24.setSurfs([crs2[3]], [0])


cc31 = cell("cc31", mod)
cc31.setSurfs([crs3[0]],[1])
cc32 = cell("cc32", clad)
cc32.setSurfs([crs3[0], crs3[1]],[0, 1])
cc33 = cell("cc33", clad)
cc33.setSurfs([crs3[2], crs3[3]],[0, 1])
cc34 = cell("cc34", mod)
cc34.setSurfs([crs3[3]], [0])

T1 = universe("T1")
T1.setGeom([cc11, cc12, cc13, cc14])

T2 = universe("T2")
T2.setGeom([cc21, cc22, cc23, cc24])

T3 = universe("T3")
T3.setGeom([cc31, cc32, cc33, cc34])


In [6]:
# area1 = 4*(span1/360)*np.pi*(roPad1*roPad1 - riPad1*riPad1)
# print(area1)

# area2 = 4*(span2/360)*np.pi*(roPad2*roPad2 - riPad2*riPad2)
# print(area2)

# area3 = 4*(span3/360)*np.pi*(roPad3*roPad3 - riPad3*riPad3)
# print(area3)

# n_g = 24 
# area1_assembly = area1*n_g
# print(area1_assembly)

# Channels
#### Fuel Rods 5 Axial layers
1. Bottom Reflector Span 20 cm
2. Bottom Axial Blanket Span 5 cm 
3. 4.95% Enriched Fuel Axial Span 185 cm
4. Top Axial Blanket Span 10 cm
5. Top Refelector Span 20 cm

#### Guide Tube 5 Axial layers
1. Bottom Reflector Span 20 cm
2. Bottom BA Cutoff Span 5 cm
3. BA integration Span 185 cm
4. Top BA Cutoff Span 10 cm
5. Top Refelector Span 20 cm

#### Instrumentation Tube 3 Axial Layers
1. Bottom Reflector Span 20 cm
2. Guide Tube 200 cm
5. Top Refelector Span 20 cm

#### Water Gap 3 Axial Layers
1. Bottom Reflector Span 20 cm
2. Water 200 cm
5. Top Refelector Span 20 cm


In [7]:
FC1 = pinStack("FC1", 0, 0, 5)
FC1.setStack(np.array([RB, F4, F1, F4, RT]), np.array([0.0, 20.0, 25.0, 210.0, 220.0]))

FC2 = FC1.duplicate("FC2")
FC2.replacePin(F1, F2)

FC3 = FC1.duplicate("FC3")
FC3.replacePin(F1, F3)

GT1 = pinStack("GT1", 0, 0, 5)
GT1.setStack(np.array([RB, G1, T1, G1, RT]), np.array([0.0, 20.0, 25.0, 210.0, 220.0]))

GT2 = GT1.duplicate("GT2")
GT2.replacePin(T1, T2)

GT3 = GT1.duplicate("GT3")
GT3.replacePin(T1, T3)

CT1 = pinStack("CT1", 0, 0, 3)
CT1.setStack(np.array([RB, G1, RT]), np.array([0.0, 20.0, 220.0]))

WG1 = CT1.duplicate("WG1")
WG1.replacePin(G1, W1)
               
latList1 = lats()
latList1.addLats([FC1, FC2, FC3, GT1, GT2, GT3, CT1, WG1])


# 2D Assembly

1. Pin pitch (cm) 1.2623




2. 17 x 17

In [8]:
FA1 = sqLat("FA1", 0, 0, 19, 1.2623)

FA1Map = np.array([
    [WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, CT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1]
])

FA1.setMap(FA1Map)
# print(FA1.elements)
# print("\n")
# print(FA1.univMats)
# print()
# print("\n")
print(FA1.toString())

lat FA1 1 0 0  19 19 1.2623
WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 
WG1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 WG1 
WG1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 WG1 
WG1 FC1 FC1 FC1 FC1 FC1 GT1 FC1 FC1 GT1 FC1 FC1 GT1 FC1 FC1 FC1 FC1 FC1 WG1 
WG1 FC1 FC1 FC1 GT1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 GT1 FC1 FC1 FC1 WG1 
WG1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 WG1 
WG1 FC1 FC1 GT1 FC1 FC1 GT1 FC1 FC1 GT1 FC1 FC1 GT1 FC1 FC1 GT1 FC1 FC1 WG1 
WG1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 WG1 
WG1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 WG1 
WG1 FC1 FC1 GT1 FC1 FC1 GT1 FC1 FC1 CT1 FC1 FC1 GT1 FC1 FC1 GT1 FC1 FC1 WG1 
WG1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 WG1 
WG1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 FC1 WG1 
WG1 FC1 FC1 GT1 FC1 FC1 GT1 FC1 FC1 GT1 FC1 FC1 

# Creating 2D Assemblies for each FA type

1. Assembly pitch (cm) 21.6038

In [9]:
pinList2 = pins()
pinList2.addPins([F1, G1, W1])

pinList3 = pins()
pinList3.addPins([F2, G1, W1])

pinList4 = pins()
pinList4.addPins([F3, G1, W1])

pinListRef = pins()
pinListRef.addPins([F1, G1, W1])

FA1_2D = FA1.duplicate("FA")

FA1_2D.replacePin(FC1, F1)
FA1_2D.replacePin(WG1, W1)
FA1_2D.replacePin(GT1, T1)
FA1_2D.replacePin(CT1, G1)

print(FA1_2D.toString())

FA2_2D = FA1_2D.duplicate("FA")
FA2_2D.replacePin(F1, F2)
FA2_2D.replacePin(T1, T2)

print(FA2_2D.toString())
FA3_2D = FA1_2D.duplicate("FA")
FA3_2D.replacePin(F1, F3)
FA3_2D.replacePin(T1, T3)


print(FA3_2D.toString())

FA_REF2D = FA1_2D.duplicate("FA")
FA_REF2D.replacePin(T1, G1)


h2D = housing(width=21.6038, height=0, defaultCRFlag = False)


lat FA 1 0 0  19 19 1.2623
W1 W1 W1 W1 W1 W1 W1 W1 W1 W1 W1 W1 W1 W1 W1 W1 W1 W1 W1 
W1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 W1 
W1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 W1 
W1 F1 F1 F1 F1 F1 T1 F1 F1 T1 F1 F1 T1 F1 F1 F1 F1 F1 W1 
W1 F1 F1 F1 T1 F1 F1 F1 F1 F1 F1 F1 F1 F1 T1 F1 F1 F1 W1 
W1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 W1 
W1 F1 F1 T1 F1 F1 T1 F1 F1 T1 F1 F1 T1 F1 F1 T1 F1 F1 W1 
W1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 W1 
W1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 W1 
W1 F1 F1 T1 F1 F1 T1 F1 F1 G1 F1 F1 T1 F1 F1 T1 F1 F1 W1 
W1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 W1 
W1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 W1 
W1 F1 F1 T1 F1 F1 T1 F1 F1 T1 F1 F1 T1 F1 F1 T1 F1 F1 W1 
W1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 W1 
W1 F1 F1 F1 T1 F1 F1 F1 F1 F1 F1 F1 F1 F1 T1 F1 F1 F1 W1 
W1 F1 F1 F1 F1 F1 T1 F1 F1 T1 F1 F1 T1 F1 F1 F1 F1 F1 W1 
W1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 F1 

# Final Core Object and Settings
1. Power = 200E6 W
2. Assemblies 37
3. Active Core Height 200 cm

In [10]:
power  = 200E6
nAssem = 37
acHeight = 200
powerAssembly = power/nAssem
powAssem2D = powerAssembly/acHeight

In [11]:
# input2 = core(mainUniv=FA_REF2D, housing=h2D, lats=None, pins=pinListRef, materials=matList, flagBurn = True, flagXS = False)
# input2.setSettings(power=powAssem2D, bc=[2], sym=0, egrid = [5e-5, 1e-9, 15.0],  nps=20000, nact=40, nskip=20, setPCC = False, 
#                    misc = ["ene 1 1 1E-11 0.625E-6 15", "include bigT_REF_2D.txt.mvol"])

input3 = core(mainUniv=FA1_2D, housing=h2D, lats = None, pins = None, materials = None, flagBurn = True, flagXS = False)
input3.setSettings(power=powAssem2D, bc=[2], egrid = [5e-5, 1e-9, 15.0],  nps=20000, nact=40, nskip=20, setPCC = False, 
                   misc = ["ene 1 1 1E-11 0.625E-6 15", "include bigT_FA1_2D.txt.mvol"])

# input4 = core(mainUniv=FA2_2D, housing=h2D, lats=None, pins=pinList3, materials=matList, flagBurn = True, flagXS = False)
# input4.setSettings(power=powAssem2D, bc=[2], sym=0, egrid = [5e-5, 1e-9, 15.0],  nps=20000, nact=40, nskip=20, setPCC = False, 
#                    misc = ["ene 1 1 1E-11 0.625E-6 15", "include bigT_FA2_2D.txt.mvol"])

# input5 = core(mainUniv=FA3_2D, housing=h2D, lats=None, pins=pinList4, materials=matList, flagBurn = True, flagXS = False)
# input5.setSettings(power=powAssem2D, bc=[2], sym=0, egrid = [5e-5, 1e-9, 15.0],  nps=20000, nact=40, nskip=20, setPCC = False, 
#                    misc = ["ene 1 1 1E-11 0.625E-6 15", "include bigT_FA3_2D.txt.mvol"])

absPath = "/mnt/c/Users/iaguirre6/Documents/xsdata/endfb7"
thermLib = "therm lwtr lwj3.11t"
# input2.setXSLib(absPath, thermLib)
input3.setXSLib(absPath, thermLib)
# input4.setXSLib(absPath, thermLib)
# input5.setXSLib(absPath, thermLib)

AttributeError: 'NoneType' object has no attribute 'id'

In [ ]:
inventory = np.array([541350, 922340, 922350, 942390])
burnPoints = np.array([
    0.10000,
    0.20000,
    0.30000,
    0.40000,
    0.50000,
    0.60000,
    0.70000,
    0.80000,
    1.00000,
    2.00000,
    4.50000,
    6.00000,
    8.50000,
    10.00000,
    12.50000,
    14.00000,
    16.50000,
    18.00000,
    20.00000,
    22.00000,
    24.00000,
    26.00000,
    28.00000,
    30.00000,
    32.00000,
    36.00000,
    40.00000,
    44.00000,
    50.00000
])
# input2.setBurnup(inventory=inventory, burnPoints=burnPoints, isDayTot=False)
input3.setBurnup(inventory=inventory, burnPoints=burnPoints, isDayTot=False)
# input4.setBurnup(inventory=inventory, burnPoints=burnPoints, isDayTot=False)
# input5.setBurnup(inventory=inventory, burnPoints=burnPoints, isDayTot=False)

In [ ]:
# input2.writeFile("bigT_REF_2D.txt")
# text_file = open("bigT_REF_2D.txt") #Display contents of created inputFile
# file_content = text_file.read()
# print(file_content)
# text_file.close()

In [ ]:
input3.writeFile("bigT_FA1_2D.txt")
text_file = open("bigT_FA1_2D.txt") #Display contents of created inputFile
file_content = text_file.read()
print(file_content)
text_file.close()

In [ ]:
# input4.writeFile("bigT_FA2_2D.txt")
# text_file = open("bigT_FA2_2D.txt") #Display contents of created inputFile
# file_content = text_file.read()
# print(file_content)
# text_file.close()

In [ ]:
# input5.writeFile("bigT_FA3_2D.txt")
# text_file = open("bigT_FA3_2D.txt") #Display contents of created inputFile
# file_content = text_file.read()
# print(file_content)
# text_file.close()


# Branching Analysis